In [1]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.3 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.15.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [8]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

  return model

In [10]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters


In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.9079999923706055

Best val_accuracy So Far: 0.9096666574478149
Total elapsed time: 00h 02m 52s


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 80)        2080      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 64)        46144     
                                                                 
 flatten (Flatten)           (None, 30976)             0         
                                                                 
 dense (Dense)               (None, 96)                2973792   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 3022986 (11.53 MB)
Trainable params: 3022986 (11.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 13s 6ms/step - loss: 0.1528 - accuracy: 0.9435 - val_loss: 0.2757 - val_accuracy: 0.9040
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1154 - accuracy: 0.9564 - val_loss: 0.2966 - val_accuracy: 0.9133
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0852 - accuracy: 0.9689 - val_loss: 0.3165 - val_accuracy: 0.9050
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0644 - accuracy: 0.9765 - val_loss: 0.3549 - val_accuracy: 0.9125
Epoch 8/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0512 - accuracy: 0.9812 - val_loss: 0.4356 - val_accuracy: 0.9112
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0393 - accuracy: 0.9856 - val_loss: 0.4873 - val_accuracy: 0.9112
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0343 - accuracy: 0.9872 - val_loss: 0.4602 - val_accurac